# Reproducing FakET experiments on SHREC21 data

## Naming convention and contents of the created files

1. `faket/class_mask.mrc` = `class_mask.mrc` sliced to valid region.
1. `faket/occupancy_mask.mrc` = `occupancy_mask.mrc` slice to valid region.
1. `reconstruction.mrc` = `faket/reconstruction_shrec.mrc` slice to valid region for tomogram 9.
1. `faket/projections_noiseless.mrc` = `grandmodel_unbinned.mrc` measured with Radon transform.
1. `faket/projections_content.mrc` = `faket/projections_noiseless.mrc` + noise (std=0.1) shifted & scaled according its style*.
1. `faket/projections_noisy.mrc` = `faket/projections_noiseless.mrc` + noise (std=0.4) shifted & scaled according its style*.
1. `faket/projections_styled.mrc` = result of NST initialized with `faket/projections_noisy.mrc`, content `faket/projections_content.mrc`, and using its style*.
1. `faket/reconstruction_content.mrc` = reconstruction of `faket/projections_content.mrc`.
1. `faket/reconstruction_noisy.mrc` = reconstruction of `faket/projections_noisy.mrc`.
1. `faket/reconstruction_styled.mrc` = reconstruction of `faket/projections_styled.mrc`.
1. `faket/reconstruction_baseline.mrc` = reconstruction of `projections.mrc`.

\* Each time we mention style in the text above, it refers to a `projections.mrc` file from a model_N+1. In case N=8, the style is taken from N=0.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
import numpy as np
import multiprocessing
import gpuMultiprocessing
from itertools import product
from os.path import join as pj
import matplotlib.pyplot as plt
from faket.data import get_clim, get_theta
from faket.data import match_mean_std, normalize
from faket.data import downsample_sinogram_space
from faket.data import slice_to_valid, vol_to_valid
from faket.data import load_mrc, save_mrc, save_conf
from faket.transform import noise_projections
from faket.transform import radon_3d, reconstruct_mrc

In [ ]:
data_folder = 'data/shrec2021_extended_dataset/'

In [ ]:
# SHREC21 provides the data in square shape even
# thought the data is stored only in the center
# The following values specify where to slice
z_valid = (0.32226, 0.67382)  # Valid range normalized

## Slicing volumes to valid voxels

In [ ]:
# slice class_mask.mrc to faket/class_mask.mrc
for N in range(10):
    vol_to_valid(data_folder, f'model_{N}', 'class_mask', z_valid, 
                 out_fname='faket/class_mask.mrc')

In [ ]:
# slice occupancy_mask.mrc to faket/occupancy_mask.mrc
for N in range(10):
    vol_to_valid(data_folder, f'model_{N}', 'occupancy_mask', z_valid, 
                 out_fname='faket/occupancy_mask.mrc')

In [ ]:
# slice reconstruction.mrc to faket/reconstruction_shrec.mrc
# for sanity-check whether we get similar results from the original challenge data
for N in range(10):
    vol_to_valid(data_folder, f'model_{N}', 'reconstruction', z_valid, 
                 out_fname='faket/reconstruction_shrec.mrc')

## Creating projections

### Noiseless

In [ ]:
# create faket/projections_noiseless.mrc by measuring the grandmodel_unbinned.mrc with Radon transform
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = {
        'input_mrc': pj(data_folder, f'model_{N}', 'grandmodel_unbinned.mrc'),
        'output_mrc': pj(data_folder, f'model_{N}', 'faket/projections_noiseless.mrc'),
        'radon_kwargs': {
            'theta': get_theta(data_folder, N),
            'dose': 0,
            'out_shape': 1024,
            'slice_axis': 1,
            # circle=False because we measure with the data outside the circle 
            # but later we cut the measurements to desired shape 
            # SHREC did it this way - confirmed from a personal communication
            'circle': False
        }
    }
    volume = load_mrc(conf['input_mrc'])
    sinogram = radon_3d(volume, **conf['radon_kwargs'])
    save_conf(conf['output_mrc'], conf)
    save_mrc(sinogram.astype(np.float32), conf['output_mrc'], overwrite=True)
print('Done')

### Noisy & Contnet

Adding Gaussian noise in projection space and matching tilt-wise mean&std of style projections

In [ ]:
# create faket/projections_content.mrc and faket/projections_noisy.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    style_N = (N + 1) % 9 # For the last train model we take style stats from the first train model
    
    # Noisy modality
    conf = {
        'input_mrc': pj(data_folder, f'model_{N}', 'faket/projections_noiseless.mrc'),
        'style_mrc': pj(data_folder, f'model_{style_N}', 'projections.mrc'),
        'output_mrc': pj(data_folder, f'model_{N}', 'faket/projections_noisy.mrc'),
        'mean': 0.0,
        'std': 0.4,
        'clip_outliers': (0.0001, 0.9999),
        'seed': N,
    }
    save_conf(conf['output_mrc'], conf)
    noise_projections(**conf)
    
    # Content modality
    conf2 = {
        'output_mrc': pj(data_folder, f'model_{N}', 'faket/projections_content.mrc'),
        'std': 0.1,
    }
    conf.update(conf2)
    save_conf(conf['output_mrc'], conf)
    noise_projections(**conf)

print('Done')

### Styled

Neural Style Transfer in projection space

In [ ]:
command_queue_nst = []

In [ ]:
nstc = {  # NEURAL STYLE TRANSFER BASE CONFIG
    # The commented params will be set later
    # 'content': 'example.mrc',
    # 'style': 'example.mrc',
    # '--init': 'example.mrc',
    # '--output': 'example.mrc', 
    # '--random-seed': None,
    '--style-weights': 1.0,  # if number of style images is 1, 1.0 is the same as None
    '--content-weight': 1.0,  # weight of the content loss relative to style loss
    '--tv-weight': 0,
    '--min-scale': 1024,
    '--end-scale': 1024,
    '--iterations': 1,
    '--initial-iterations': 1,
    '--save-every': 2,
    '--step-size': 0.15,
    '--avg-decay': 0.99,
    '--style-scale-fac': 1.0,
    '--pooling': 'max',
    '--devices': 'cuda:0', #
    '--seq_start' : 0,
    '--seq_end' : 61,
    '--content_layers': '8 11',
    '--content_layers_weights': '67 33'
}

def get_command(expname, nst_command, config):
    command = (
    f"EXPNAME={expname} {nst_command} "
    f"{config['content']} {config['style']} "
    f"{' '.join([f'{k} {v}' for k, v in config.items() if k.startswith('--')])}")
    return command

In [ ]:
# create faket/projections_styled.mrc
gpu_id_list = [0, 4, 5, 6, 7]
NST_command = 'PYTHONHASHSEED=0 python3 -m faket.style_transfer.cli'

# command_queue_nst = []
for N in range(0, 9): # We do not need this modality for the test model_9
    style_N = (N + 1) % 9 # For the last train model we take style stats from the first train model
    
    EXPNAME = f'TOMOGRAM_{N}'  # Just for visualizing the progress
    tomo_folder = pj(data_folder, f'model_{N}', 'faket')

    conf = nstc.copy()
    conf.update({
        'content': pj(tomo_folder, 'projections_content.mrc'),
        'style': pj(data_folder, f'model_{style_N}', 'projections.mrc'), 
        '--init': pj(tomo_folder, 'projections_noisy.mrc'),
        '--output': pj(tomo_folder, 'projections_styled.mrc'), 
        '--random-seed': N,
    })
    
    command = get_command(EXPNAME, NST_command, conf)
    command_queue_nst.append(command)

In [ ]:
# Save the command queue to a file
with open('command_queue_nst.txt', 'w') as fl:
    fl.writelines('\n'.join(command_queue_nst))

In [ ]:
# Run all the commands (returns list of failed commands if any)
gpuMultiprocessing.queue_runner(command_queue_nst, gpu_id_list,
                                env_gpu_name='CUDA_VISIBLE_DEVICES',
                                processes_per_gpu=6, allowed_restarts=1)

## Computing reconstructions

In [ ]:
recc = {  # RECONSTRUCTION BASE CONFIG
    'downsample_angle' : 1,  # Sinogram downsampling in theta dimension (1 = no downsampling)
    'downsample_pre' : 2,  # Sinogram downsampling (1 = no downsampling)
    'order' : 3,  # Downsampling in space with spline interpolation of order (0 - 5)
    'filtering' : 'ramp2d',  # Filter used during reconstruction in FBP algorithm
    'filterkwargs' : {'crowtherFreq': 25, 'radiusCutoff': 230, 'angularCutoff': (0, 83)},
    'downsample_post' : 1,  # Reconstruction downsampling
    'ncpus': 61, # multiprocessing.cpu_count(),  # Number of CPUs to use while reconstructing
    'z_valid': z_valid # 2-tuple range of valid pixels in Z dimension normalized from 0 to 1. (0., 1.) or None for all.
}

### Noiseless

In [ ]:
# reconstruct faket/projections_noiseless.mrc to produce faket/reconstruction_noiseless.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' :  pj(data_folder, f'model_{N}', 'faket/projections_noiseless.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', 'faket/reconstruction_noiseless.mrc')
    })
    reconstruct_mrc(**conf)

### Baseline

In [ ]:
# reconstruct projections.mrc to produce faket/reconstruction_baseline.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' :  pj(data_folder, f'model_{N}', 'projections.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', 'faket/reconstruction_baseline.mrc')
    })
    reconstruct_mrc(**conf)

### Content

In [ ]:
# reconstruct faket/projections_content.mrc to produce faket/reconstruction_content.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' :  pj(data_folder, f'model_{N}', 'faket/projections_content.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' :  pj(data_folder, f'model_{N}', 'faket/reconstruction_content.mrc')
    })
    reconstruct_mrc(**conf)

### Noisy

In [ ]:
# reconstruct faket/projections_noisy.mrc to produce faket/reconstruction_noisy.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : pj(data_folder, f'model_{N}', 'faket/projections_noisy.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', 'faket/reconstruction_noisy.mrc')
    })
    reconstruct_mrc(**conf)

### Styled

In [ ]:
# reconstruct faket/projections_styled.mrc to produce faket/reconstruction_styled.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : pj(data_folder, f'model_{N}', f'faket/projections_styled.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', f'faket/reconstruction_styled.mrc.mrc')
    })
    reconstruct_mrc(**conf)

# Deep Finder experiments

Train for 30 epochs on 9 tomograms, eval on validation (last training) tomogram at every epoch

1. `DF('faket/reconstruction_baseline.mrc')` 
2. `DF('faket/reconstruction_content.mrc')`
3. `DF('faket/reconstruction_noisy.mrc')`
4. `DF('faket/reconstruction_styled.mrc')`

## Training

In [ ]:
def get_full_DF_training_command(DF_training_command, config):
    command = (
        f"{DF_training_command} "
        f"--training_tomogram_ids {' '.join(list(zip(*config['training_tomograms']))[0])} "
        f"--training_tomograms {' '.join(list(zip(*config['training_tomograms']))[1])} "
        f"{' '.join([f'{k} {v}' for k, v in config.items() if k.startswith('--')])} "       
    )
    return command

command_queue_training = []

In [ ]:
# Run the training (each job runs on one GPU, but more GPUs can run more jobs in parallel)
gpu_id_list = [0, 1, 2, 3, 4, 5]
DF_training_command ='PYTHONHASHSEED=0 python3 faket/deepfinder/launch_training.py'

# create config files from dict using json for different seeds
for idf in ['shrec']: #['_CL[6]_CLW[100]', '_CL[8]_CLW[100]', '_CL[11]_CLW[100]', '_CL[8,11]_CLW[67,33]', '_CL[8,11]_CLW[33,67]']:

    experiment_names = [f'exp_baseline_{idf}']
    training_tomograms = [f'{idf}']

    num_seeds = 10
    num_epochs = 30

    # command_queue = []
    for experiment_name, training_tomogram in zip(experiment_names, training_tomograms):
        for N in range(1, num_seeds + 1):
            training_conf = {
                "--training_tomo_path": data_folder,
                "training_tomograms": [[str(i), training_tomogram] for i in range(0,9)],
                "--num_epochs": num_epochs,
                "--out_path": pj('data', 'results', experiment_name, f'seed{N}'),
                "--save_every": 1, 
                "--seed": N,
                # If continue_training_path is the same as out_path - continue from last epoch.
                # If it is a path to a specific weights.h5 file, continue from there.
                "--continue_training_path": pj('data', 'results', experiment_name, f'seed{N}'),

            }
            command = get_full_DF_training_command(DF_training_command, training_conf)
            command_queue_training.append(command)

In [ ]:
# Save the command queue to a file
with open('command_queue_training_baseline_shrec.txt', 'w') as fl:
    fl.writelines('\n'.join(command_queue_training))

In [ ]:
# Run all the commands (returns list of failed commands if any)
gpuMultiprocessing.queue_runner(command_queue, gpu_id_list,
                                env_gpu_name='CUDA_VISIBLE_DEVICES',
                                processes_per_gpu=2, allowed_restarts=0)

## Segmentation

In [ ]:
def get_full_DF_analysis_command(DF_analysis_command, config):
    command=(
        f"{DF_analysis_command} "
        f"{' '.join([f'{k} {v}' for k, v in config.items() if k.startswith('--')])}"
    )
    return command

command_queue_segmentation = []

In [ ]:
# Run the segmentation (each job runs on one GPU, but more GPUs can run more jobs in parallel)
gpu_id_list = [0, 4, 5, 6, 7]

DF_segmentation_command ='PYTHONHASHSEED=0 python3 faket/deepfinder/launch_segmentation.py'

#experiment_names = ['exp_volnoisy']
seed_ids = range(1, 11)
num_epochs = range(1, 31)
test_tomograms = ['shrec']
test_tomo_idx = 9

for idf in ['shrec']: #['_CL[6]_CLW[100]', '_CL[8]_CLW[100]', '_CL[11]_CLW[100]', '_CL[8,11]_CLW[67,33]', '_CL[8,11]_CLW[33,67]']:
    experiment_names = [f'exp_baseline_{idf}']#[f'exp_styled{idf}']

    # command_queue_segmentation = []
    for N, num_epoch, test_tomogram, experiment_name in \
        product(seed_ids, num_epochs, test_tomograms, experiment_names):
        analysis_conf ={
            "--test_tomo_path" : data_folder,
            "--test_tomo_idx" : test_tomo_idx, 
            "--test_tomogram" : test_tomogram,
            "--num_epochs" : num_epoch,
            "--DF_weights_path" : pj('data', 'results', experiment_name, f'seed{N}'),
            "--out_path" : pj('data', 'results', experiment_name, f'seed{N}'), 
        }
        command = get_full_DF_analysis_command(DF_segmentation_command, analysis_conf)
        command_queue_segmentation.append(command)

In [ ]:
# Save the command queue to a file
with open('command_queue_segmentation_baseline_shrec.txt', 'w') as fl:
    fl.writelines('\n'.join(command_queue_segmentation))

In [ ]:
# Run all the commands (returns list of failed commands if any)
gpuMultiprocessing.queue_runner(command_queue_segmentation, gpu_id_list,
                                env_gpu_name='CUDA_VISIBLE_DEVICES',
                                processes_per_gpu=2, allowed_restarts=0)

## Clustering & Evaluation

In [ ]:
# Before running the evaluation, make sure that `particle_locations.txt` file is present in the test tomogram folder.
for model in ['model_8', 'model_9']:
    src = pj(data_folder, model, 'particle_locations.txt')
    dst = pj(data_folder, model, 'faket', 'particle_locations.txt')
    shutil.copy(src, dst)

In [ ]:
command_queue_clustering = []
command_queue_evaluation = []

In [ ]:
# Compute the commands and store them in queue for clustering and evaluation
DF_clustering_command ='python3 faket/deepfinder/launch_clustering.py'
DF_evaluation_command ='python3 faket/deepfinder/launch_evaluation.py'
# experiment_names = ["exp_volnoisy"]
seed_ids = range(1, 11)
num_epochs = range(1, 31)
test_tomograms = ['shrec']
test_tomo_idx = 9

for idf in ['shrec']: #['_CL[6]_CLW[100]', '_CL[8]_CLW[100]', '_CL[11]_CLW[100]', '_CL[8,11]_CLW[67,33]', '_CL[8,11]_CLW[33,67]']:
    experiment_names = [f'exp_baseline_{idf}']
    for N, num_epoch, test_tomogram, experiment_name \
        in product(seed_ids, num_epochs, test_tomograms, experiment_names):
        analysis_conf ={
            "--test_tomogram" : test_tomogram,
            "--test_tomo_idx" : test_tomo_idx,
            "--num_epochs" : num_epoch,
            "--label_map_path" : pj('data', 'results', experiment_name, f'seed{N}'),
            "--out_path" : pj('data', 'results', experiment_name, f'seed{N}'), 
        }

        command_clustering = get_full_DF_analysis_command(DF_clustering_command, analysis_conf)
        command_queue_clustering.append(command_clustering)

        command_evaluation = get_full_DF_analysis_command(DF_evaluation_command, analysis_conf)
        command_queue_evaluation.append(command_evaluation)

In [ ]:
# Save the command queue to a file
with open('command_queue_clustering_baseline_shrec.txt', 'w') as fl:
    fl.writelines('\n'.join(command_queue_clustering))
    
# Save the command queue to a file
with open('command_queue_evaluation_baseline_shrec.txt', 'w') as fl:
    fl.writelines('\n'.join(command_queue_evaluation))

In [ ]:
# Run the clustering (each job runs on one CPU, but more CPUs can run more jobs in parallel)
num_cpus = 100
cpu_id_list = list(range(num_cpus))

gpuMultiprocessing.queue_runner(command_queue_clustering, cpu_id_list,
                                env_gpu_name='CPUID',
                                processes_per_gpu=2, allowed_restarts=0)

In [ ]:
# Run the evaluation (each job runs on one CPU, but more CPUs can run more jobs in parallel)
num_cpus = 100
cpu_id_list = list(range(num_cpus))

gpuMultiprocessing.queue_runner(command_queue_evaluation, cpu_id_list,
                                env_gpu_name='CPUID',
                                processes_per_gpu=2, allowed_restarts=0)